In [ ]:
import re
import os
from langchain_community.document_loaders import TextLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_qdrant import QdrantVectorStore
from langchain.text_splitter import (
    MarkdownHeaderTextSplitter,
    RecursiveCharacterTextSplitter
)
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from typing import List, Dict
from flashrank import Ranker, RerankRequest

from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
class Config:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") 
    QDRANT_URL = "http://localhost:6333"  
    COLLECTION_NAME = "parkinsons_faq"
    EMBEDDING_MODEL = "models/gemini-embedding-001"
    LLM_MODEL = "gemini-2.5-flash"

In [3]:
loader = TextLoader("data/processed/cleaned-parkinsons-faq.md")
raw_doc = loader.load()[0]

In [4]:
def chunk_document(cleaned_content):
    headers_to_split_on = [
        ("#", "Chapter"),
        ("##", "Section"),
        ("###", "Question")
    ]
    
    # Split only at chapter/section/question levels
    text_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        strip_headers=True,
        return_each_line=False,
    )
    
    documents = text_splitter.split_text(cleaned_content)
    
    # Process each chunk to extract Q&A format
    processed_docs = []
    for doc in documents:
        metadata = doc.metadata
        content = doc.page_content
        
        # For question chunks
        if "Question" in metadata:
            # Extract answer content including subsections and notes
            answer = re.sub(r'^#{4,5} ?', '', content, flags=re.MULTILINE)
            answer = re.sub(r'\n{3,}', '\n\n', answer).strip()
            
        processed_docs.append(Document(
            page_content=answer,
            metadata={
                "Chapter": metadata.get("Chapter", ""),
                "Section": metadata.get("Section", ""),
                "Question": metadata["Question"],
                "Answer": answer  # 🆕 add answer into metadata
            }
        ))
    
    return processed_docs


In [5]:
with open("data/processed/cleaned-parkinsons-faq.md", "r") as f:
    cleaned_content = f.read()

In [6]:
documents = chunk_document(cleaned_content)

embeddings = GoogleGenerativeAIEmbeddings(
    api_key=Config.GOOGLE_API_KEY,
    model=Config.EMBEDDING_MODEL
)

vector_store = QdrantVectorStore.from_documents(
    documents,
    embeddings,
    url=Config.QDRANT_URL,
    collection_name=Config.COLLECTION_NAME,
    force_recreate=True
)

In [9]:
ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2", cache_dir="/tmp")

In [10]:
def search_rerank(vector_store, query: str, k: int = 3, initial_k: int = 20) -> List[dict]:
    """
    Perform semantic search with FlashRank reranking.
    
    Args:
        vector_store: The vector store for initial retrieval
        query: Search query
        k: Final number of results to return
        initial_k: Number of initial results to retrieve before reranking
    """
    # Step 1: Get more results initially for reranking
    results = vector_store.similarity_search_with_score(query, k=initial_k)

    # Step 2: Prepare data for FlashRank
    passages = []
    doc_metadata = []
    
    for doc, score in results:
        # Create passage text combining question and answer for better reranking
        passage_text = f"Question: {doc.metadata.get('Question', '')}\nAnswer: {doc.page_content.strip()}"
        passages.append({
            "id": len(passages),
            "text": passage_text,
            "meta": {
                "question": doc.metadata.get("Question", ""),
                "answer": doc.page_content.strip(),
                "chapter": doc.metadata.get("Chapter", ""),
                "section": doc.metadata.get("Section", ""),
                "original_score": score
            }
        })
        doc_metadata.append((doc, score))

    # Step 3: Rerank using FlashRank
    if passages:
        rerank_request = RerankRequest(query=query, passages=passages)
        reranked_results = ranker.rerank(rerank_request)
        
        # Step 4: Apply exact match boosting after reranking
        final_results = []
        exact_matches = []
        
        for result in reranked_results[:k*2]:  # Get more candidates for exact match boosting
            passage_meta = result["meta"]
            question = passage_meta["question"].lower()
            
            reranked_item = {
                "question": passage_meta["question"],
                "answer": passage_meta["answer"],
                "chapter": passage_meta["chapter"],
                "section": passage_meta["section"],
                "original_score": passage_meta["original_score"],
                "rerank_score": result["score"],
                "combined_score": result["score"]  # Will be boosted if exact match
            }
            
            # Boost exact matches
            if query.strip().lower() == question:
                reranked_item["combined_score"] = result["score"] + 1.0  # Boost by 1.0
                exact_matches.append(reranked_item)
            else:
                final_results.append(reranked_item)
        
        # Combine exact matches (first) with reranked results
        final_results = exact_matches + final_results
        return final_results[:k]
    
    return []


In [11]:
def search(vector_store, query: str, k: int = 3) -> List[dict]:
    """
    Original semantic search function (kept for comparison).
    """
    results = vector_store.similarity_search_with_score(query, k=10)

    # Boost exact match
    boosted = []
    for doc, score in results:
        question = doc.metadata.get("Question", "").lower()
        if query.strip().lower() == question:
            boosted.insert(0, (doc, score))  # put exact match on top
        else:
            boosted.append((doc, score))

    boosted = boosted[:k]

    return [
        {
            "question": doc.metadata.get("Question", ""),
            "answer": doc.page_content.strip(),
            "chapter": doc.metadata.get("Chapter", ""),
            "section": doc.metadata.get("Section", ""),
            "score": score,
        }
        for doc, score in boosted
    ]

In [12]:
llm = ChatGoogleGenerativeAI(
    model=Config.LLM_MODEL,
    google_api_key=Config.GOOGLE_API_KEY,
    temperature=0.1
)

In [13]:
# Create the prompt template
rag_prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="""You are a helpful medical assistant specializing in Parkinson's disease. Answer the user's question based on the provided context from medical FAQs.

Instructions:
- The provided information from the context is from our retrieval results, use the information provided to help answer the question
- No need to mention the source of the information
- If the context doesn't contain enough information to answer the question, say so clearly
- Be accurate and informative
- Use a compassionate and professional tone
- If relevant, mention consulting with healthcare professionals

Context from Parkinson's Disease FAQ:
{context}

Question: {question}

Answer:"""
)

def format_context(search_results: List[dict]) -> str:
    """Format search results into context string for the prompt."""
    context = ""
    for i, result in enumerate(search_results, 1):
        context += f"\n--- FAQ Entry {i} ---\n"
        context += f"Question: {result['question']}\n"
        context += f"Answer: {result['answer']}\n"
        context += f"Source: {result['chapter']} > {result['section']}\n"
    return context.strip()

In [19]:
def generate_multiple_answers(query: str, search_results: List[dict], num_answers: int = 3) -> List[dict]:
    """
    Generate multiple answer candidates using different context combinations.
    
    Args:
        query: User's question
        search_results: Retrieved and reranked documents
        num_answers: Number of answer candidates to generate
        
    Returns:
        List of answer candidates with metadata
    """
    answer_candidates = []
    
    # Strategy 1: Use top result only (most relevant)
    if len(search_results) >= 1:
        context = format_context(search_results[:1])
        prompt = rag_prompt_template.format(question=query, context=context)
        response = llm.invoke(prompt)
        answer_candidates.append({
            "answer": response.content,
            "strategy": "single_best",
            "sources_used": search_results[:1],
            "context": context
        })
    
    # Strategy 2: Use top 2 results (balanced relevance)
    if len(search_results) >= 2:
        context = format_context(search_results[:2])
        prompt = rag_prompt_template.format(question=query, context=context)
        response = llm.invoke(prompt)
        answer_candidates.append({
            "answer": response.content,
            "strategy": "top_two",
            "sources_used": search_results[:2],
            "context": context
        })
    
    # Strategy 3: Use all retrieved results (comprehensive)
    if len(search_results) >= 3:
        context = format_context(search_results)
        prompt = rag_prompt_template.format(question=query, context=context)
        response = llm.invoke(prompt)
        answer_candidates.append({
            "answer": response.content,
            "strategy": "comprehensive",
            "sources_used": search_results,
            "context": context
        })
    
    return answer_candidates[:num_answers]

def rerank_qa_answers(query: str, answer_candidates: List[dict]) -> List[dict]:
    """
    Rerank generated answer candidates using FlashRank.
    
    Args:
        query: Original user question
        answer_candidates: List of generated answer candidates
        
    Returns:
        Reranked list of answer candidates
    """
    if not answer_candidates:
        return []
    
    # Prepare passages for reranking
    passages = []
    for i, candidate in enumerate(answer_candidates):
        passages.append({
            "id": i,
            "text": candidate["answer"],
            "meta": candidate
        })
    
    # Rerank the answers
    rerank_request = RerankRequest(query=query, passages=passages)
    reranked_results = ranker.rerank(rerank_request)
    
    # Return reranked candidates with scores
    reranked_candidates = []
    for result in reranked_results:
        candidate = result["meta"].copy()
        candidate["qa_rerank_score"] = result["score"]
        reranked_candidates.append(candidate)
    
    return reranked_candidates

def format_context(search_results: List[dict]) -> str:
    """Format search results into context string for the prompt."""
    context = ""
    for i, result in enumerate(search_results, 1):
        context += f"\n--- FAQ Entry {i} ---\n"
        context += f"Question: {result['question']}\n"
        context += f"Answer: {result['answer']}\n"
        context += f"Source: {result['chapter']} > {result['section']}\n"
        # Add rerank score if available
        if 'rerank_score' in result:
            context += f"Relevance Score: {result['rerank_score']:.4f}\n"
    return context.strip()

In [20]:
def rag_rerank(query: str, k: int = 3, use_qa_rerank: bool = True) -> dict:
    """
    Enhanced RAG pipeline with FlashRank reranking for both retrieval and QA.
    
    Args:
        query: User's question
        k: Number of search results to use as context
        use_qa_rerank: Whether to use QA answer reranking
        
    Returns:
        Dictionary containing the final answer and metadata
    """
    # Step 1: Retrieve and rerank documents
    search_results = search_rerank(vector_store, query, k=k, initial_k=20)
    
    if not use_qa_rerank:
        # Simple single answer generation
        context = format_context(search_results)
        prompt = rag_prompt_template.format(question=query, context=context)
        response = llm.invoke(prompt)
        
        return {
            "question": query,
            "answer": response.content,
            "sources": search_results,
            "context_used": context,
            "method": "retrieval_rerank_only"
        }
    
    # Step 2: Generate multiple answer candidates
    answer_candidates = generate_multiple_answers(query, search_results, num_answers=3)
    
    # Step 3: Rerank answer candidates
    reranked_answers = rerank_qa_answers(query, answer_candidates)
    
    # Step 4: Select best answer
    best_answer = reranked_answers[0] if reranked_answers else answer_candidates[0]
    
    return {
        "question": query,
        "answer": best_answer["answer"],
        "best_strategy": best_answer["strategy"],
        "qa_rerank_score": best_answer.get("qa_rerank_score", 0),
        "sources": search_results,
        "all_candidates": reranked_answers,
        "context_used": best_answer["context"],
        "method": "full_rerank_pipeline"
    }

In [21]:
def rag(query: str, k: int = 3) -> dict:
    """
    Original RAG pipeline (kept for comparison).
    """
    # Step 1: Retrieve relevant documents
    search_results = search(vector_store, query, k=k)
    
    # Step 2: Format context
    context = format_context(search_results)
    
    # Step 3: Generate prompt
    prompt = rag_prompt_template.format(question=query, context=context)
    
    # Step 4: Get LLM response
    response = llm.invoke(prompt)
    
    # Step 5: Return structured result
    return {
        "question": query,
        "answer": response.content,
        "sources": search_results,
        "context_used": context,
        "method": "original_pipeline"
    }


In [24]:
print("=" * 80)
print("COMPARING ORIGINAL vs RERANKED PIPELINES")
print("=" * 80)

comparison_queries = [
    "What medications are available for Parkinson's?",
    "Can Parkinson's disease be cured?",
    "What are the main symptoms of Parkinson's?"
]
for query in comparison_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    # Original pipeline
    print(f"\n🔵 ORIGINAL PIPELINE:")
    original_result = rag(query, k=3)
    print(f"Answer: {original_result['answer']}")
    print(f"Sources: {[s['question'] for s in original_result['sources']]}")
    
    # Reranked pipeline (retrieval rerank only)
    print(f"\n🟢 RETRIEVAL RERANK ONLY:")
    rerank_simple = rag_rerank(query, k=3, use_qa_rerank=False)
    print(f"Answer: {rerank_simple['answer']}")
    print(f"Sources: [{', '.join([f'{s['question']}(R:{s.get('rerank_score', 0)})' for s in rerank_simple['sources']])}]")
    
    
    print("\n" + "-" * 60)

COMPARING ORIGINAL vs RERANKED PIPELINES

Query: What medications are available for Parkinson's?

🔵 ORIGINAL PIPELINE:


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/parkinsons_faq/points/query "HTTP/1.1 200 OK"


Answer: There are many effective prescription medications available to help manage Parkinson's symptoms. Treatment is often tailored to an individual's unique needs, and it's common for people to take a variety of medications at different doses and times to manage their symptoms effectively.

Most Parkinson's medications work by replacing, mimicking, or boosting dopamine, a brain chemical crucial for mood, movement, and motivation. These medications are associated with potential side effects, and the choice of medication and dose is influenced by symptoms, side effects, and how well it helps you continue your daily activities. Medications work best when taken on a regular schedule and combined with exercise, good nutrition, and adequate sleep.

Here are some of the medications currently available:

*   **Levodopa:** This is considered the most potent medication for Parkinson's. It converts into dopamine in the brain. To minimize side effects like nausea and vomiting, it's usually combi

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/parkinsons_faq/points/query "HTTP/1.1 200 OK"


Answer: There are many effective prescription medications available to help manage Parkinson's symptoms. It's common for individuals to take a combination of these medications at different doses and times throughout the day.

Most Parkinson's medications work by replacing, mimicking, or boosting dopamine, a crucial brain chemical involved in mood, movement, and motivation. It's important to be aware that dopamine-related drugs can have potential side effects such as nausea, drowsiness, low blood pressure, hallucinations, dyskinesia, and compulsive behaviors.

Here are some of the types of medications commonly used:

*   **Levodopa:** This is considered the most potent medication for Parkinson's. It's often combined with carbidopa to reduce nausea and vomiting. Levodopa comes in various forms (immediate-release, controlled-release, continuous delivery, or for "off" episodes). It's crucial to keep track of the specific type and strength you take, as accidental substitutions can affect sy

...

Answer: While there is currently no cure for Parkinson's disease, researchers are making significant breakthroughs in understanding the disease, its causes, and how to further improve therapies. Many Parkinson's symptoms can be effectively treated.

The goal of current treatments is to improve movement and relieve symptoms with as few side effects as possible. Treatment plans are highly individualized and can include medications, exercise, rehabilitation therapies (physical, occupational, and speech), and sometimes surgery.

Comprehensive research is ongoing to unlock the causes of Parkinson's, develop life-changing treatments, and ultimately discover a cure. Organizations are investing in innovative science, funding researchers, and sharing findings to drive progress in the fight for a cure.

It's always a good idea to discuss your specific situation and treatment options with your healthcare professional.
Sources: [Can Parkinson's be cured?(R:0.9999486207962036), What are the future 

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/parkinsons_faq/points/query "HTTP/1.1 200 OK"


Answer: Parkinson's disease (PD) is characterized by a range of symptoms, which can be broadly categorized into movement (motor) and non-movement (non-motor) symptoms.

The main movement symptoms, which are often the most noticeable, include:

*   **Slow movements (Bradykinesia):** This is a slowing of spontaneous and automatic movements and must be present for a Parkinson's diagnosis. It can manifest as slower walking, limited arm swing, or decreased blinking and facial expression.
*   **Stiffness (Rigidity):** This refers to stiffness in the arms or legs that goes beyond what might be expected from normal aging or arthritis. It can affect movement, cause discomfort and pain, and may interfere with sleep.
*   **Resting Tremor:** In early Parkinson's, about 70% of people experience a slight tremor, often in a hand or foot on one side of the body, or less commonly in the jaw or face. This tremor usually appears when muscles are relaxed or at rest and can worsen with stress.
*   **Balanc

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/parkinsons_faq/points/query "HTTP/1.1 200 OK"


Answer: Parkinson's disease (PD) is characterized by a range of symptoms, which can be broadly categorized into movement and non-movement symptoms.

The more noticeable movement symptoms include:
*   **Tremor:** Involuntary shaking.
*   **Stiffness:** Rigidity in the limbs and trunk.
*   **Slow movements (bradykinesia):** Difficulty initiating movement and slowness in performing activities.
*   **Balance issues:** Problems with stability and coordination.

In addition to these, non-movement symptoms are also very common and can be quite challenging. These may include:
*   Trouble sleeping
*   Depression
*   Speech problems

It's important to remember that symptoms can vary from person to person. If you have concerns about symptoms, it's always best to consult with healthcare professionals for an accurate diagnosis and personalized advice.
Sources: [What is Parkinson's disease?(R:0.9991752505302429), What is Parkinson's vs. parkinsonism?(R:0.998681366443634), Can stress worsen Parkinson